In [4]:
import json
import requests
import pandas as pd
pd.set_option('display.max_rows', None)

def extract_works_data(url):
    response = requests.get(url)
    data = json.loads(response.content.decode('utf-8'))

    records = []

    for group in data.get('groups', []):
        year = None
        title = ''
        works = group.get('works', [])

        for work in works:
            if year is None:
                pub_date = work.get('publicationDate')
                if pub_date:
                    yr = pub_date.get('year')
                    if yr:
                        year = int(yr)
            t = work.get('title', {})
            if isinstance(t, dict):
                t_val = t.get('value', '')
            else:
                t_val = str(t) if t else ''
            if len(t_val) > len(title):
                title = t_val

        doi = None
        eid = None

        for ext_id in group.get('externalIdentifiers', []):
            id_type = ext_id.get('externalIdentifierType', {}).get('value', '').lower()
            id_value = ext_id.get('externalIdentifierId', {}).get('value')
            if id_type == 'doi' and doi is None:
                doi = id_value
            elif id_type == 'eid' and eid is None:
                eid = id_value

        records.append({
            'Год издания': year,
            'Название работы': title if title else None,
            'Идентификатор doi': doi,
            'Идентификатор eid': eid
        })

    df = pd.DataFrame(records)

    def validate_df(df):
        assert len(data['groups']) == len(df), "Количество записей не совпадает"
        assert (len(df) - df['Идентификатор doi'].nunique() - df['Идентификатор doi'].isna().sum()) == 0, "DOI не уникальны"
        assert (len(df) - df['Идентификатор eid'].nunique() - df['Идентификатор eid'].isna().sum()) == 0, "EID не уникальны"
        assert (df['Идентификатор doi'].notna().sum() - df['Идентификатор doi'].str.startswith('10.').sum()) == 0, "Некорректный формат DOI"
        assert (df['Идентификатор eid'].notna().sum() - df['Идентификатор eid'].str.startswith('2').sum()) == 0, "Некорректный формат EID"
        assert df['Год издания'].isna().sum() == 0, "Есть пропуски в годе издания"
        assert df['Название работы'].isna().sum() == 0, "Есть пропуски в названиях работ"
        years = df['Год издания'].dropna()
        assert years.between(1994, 2026).all(), "Годы вне допустимого диапазона"
        print('Данные выгружены корректно!')

    validate_df(df)
    print(f'\nКоличество работ ученого: {len(data["groups"])}')
    print(f'Выгружено работ: {len(records)}')

    return df


df = extract_works_data('https://orcid.org/0000-0002-8520-7267/allWorks.json?sort=date&sortAsc=false')

df['Год издания'] = pd.to_numeric(df['Год издания'], errors='coerce').astype('Int64')

# ИСПРАВЛЕНО: упрощён подсчёт работ без идентификаторов
no_ind_list = df[df['Идентификатор doi'].isna() & df['Идентификатор eid'].isna()].index.tolist()
print(f'\n- Работ без идентификаторов doi и eid: {len(no_ind_list)}')
print(f'- Индексы работ без doi и eid:\n{no_ind_list}')

print('\nТаблица:')
display(df)

df.to_excel('список_работ.xlsx', index=False)

Данные выгружены корректно!

Количество работ ученого: 711
Выгружено работ: 711

- Работ без идентификаторов doi и eid: 30
- Индексы работ без doi и eid:
[204, 205, 206, 207, 233, 235, 237, 238, 240, 242, 291, 292, 295, 380, 381, 382, 383, 385, 409, 426, 427, 428, 470, 500, 507, 517, 541, 542, 600, 621]

Таблица:


,Год издания,Название работы,Идентификатор doi,Идентификатор eid
0,2026,Deep Unfolding of Iterative Precoding-Based Nu...,10.1109/TCOMM.2025.3642692,None
1,2026,Resource Allocation Schemes for Scalable Panel...,10.1109/OJVT.2026.3652908,None
2,2025,On Deep Learning Hybrid Architectures for MIMO...,10.3390/electronics14234692,2-s2.0-105024537957
3,2025,Data-Oriented Channel Knowledge Map IoT Transm...,10.1109/VTC2025-Spring65109.2025.11174660,2-s2.0-105019047127
4,2025,Newmann Series-Based Precoding Weight Design f...,10.1109/VTC2025-Spring65109.2025.11174837,2-s2.0-105019051188
5,2025,Genetic Resource Allocation Algorithm for Pane...,10.3390/electronics14153107,None
6,2025,Position Accuracy and Distributed Beamforming ...,10.3390/fi17060236,None
7,2025,Neural-Network-Based Interference Cancellation...,10.3390/electronics14102083,2-s2.0-105006664111
8,2025,Neural Network-Based Interference Cancellation...,10.20944/preprints202504.1155.v1,None
9,2025,"Improved Low-Complexity, Pilot-Based Channel E...",10.3390/app15073743,2-s2.0-105002277915
